# Important note!

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your GT login and the GT logins of any of your collaborators below. (The GT logins are worth 1 point per notebook, so don't miss the opportunity to get a free point!)

In [ ]:
YOUR_ID = "" # Please enter your GT login, e.g., "rvuduc3" or "gtg911x"
COLLABORATORS = [] # list of strings of your collaborators' IDs

In [ ]:
import re

RE_CHECK_ID = re.compile (r'''[a-zA-Z]+\d+|[gG][tT][gG]\d+[a-zA-Z]''')
assert RE_CHECK_ID.match (YOUR_ID) is not None

collab_check = [RE_CHECK_ID.match (i) is not None for i in COLLABORATORS]
assert all (collab_check)

del collab_check
del RE_CHECK_ID
del re

**Jupyter / IPython version check.** The following code cell verifies that you are using the correct version of Jupyter/IPython.

In [ ]:
import IPython
assert IPython.version_info[0] >= 3, "Your version of IPython is too old, please update it."

# Interacting agents

Recall the Romeo and Juliet model, which had two "agents" that reacted to one another. Any model or simulation where there are interacting agents are sometimes referred to as _agent-based models_. This notebook introduces another example of an agent-based model known as the _aggressor-defender_ model.

In [ ]:
import numpy as np
import scipy as sp

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

Let's generate a set of `N` points, distributed uniformly at random in the 2-D domain $[0, 1.0] \times [0, 1.0]$.

In [ ]:
N = 10
points_0 = np.random.uniform (low=0.0, high=1.0, size=(N, 2))
print (points_0)

Here is some code to plot these points.

In [ ]:
IX, IY = 0, 1

def plot_points (points):
    plt.scatter (points[:, IX], points[:, IY], s=100.0)
    plt.axis ('square')
    plt.axis ([0.0, 1.0, 0.0, 1.0])
    
def plot_points_triplet (lpoints, ltitle, mpoints, mtitle, rpoints, rtitle):
    plt.figure (figsize=(15, 5))
    plt.subplot (1, 3, 1)
    plot_points (lpoints)
    plt.title (ltitle)
    plt.subplot (1, 3, 2)
    plot_points (mpoints)
    plt.title (mtitle)
    plt.subplot (1, 3, 3)
    plot_points (rpoints)
    plt.title (rtitle)

plot_points (points_0)

Consider an index set $I = \{0, 1, 2, \ldots, n-1\}$. For each $i \in I$, the following function selects a pair of "partners" $(a_i, b_i)$ where $a_i \neq b_i$ and $a_i, b_i \in I - \{i\}$.

In [ ]:
def select_partners (n):
    ids = np.arange (n)
    partners = np.zeros ((n, 2), dtype=int)
    for i in range (n):
        # Swap out i
        ids[i], ids[-1] = ids[-1], ids[i]
        
        # Select partners
        partners[i, :] = np.random.choice (ids[:-1], 2, replace=False)
        
        # Swap i back in
        ids[i], ids[-1] = ids[-1], ids[i]
    return partners

partners = select_partners (N)
print (partners)

**Exercise 1** (1 point). Complete the function, `step_chicken()`, which implements the "chicken" behavior. That is, consider a point $i$ whose partners are $(a_i, b_i)$. When $i$ is a chicken, then $a_i$ is the "aggressor" and $b_i$ is the "defender"; $i$ tries to move itself so that $b_i$ is between it and $a_i$.

In [ ]:
DELTA_T = 0.05 # Some "time-step" constant

def wrap (points):
    n = len (points)
    for i in range (n):
        if points[i, 0] < 0:
            points[i, 0] += 1.0
        elif points[i, 0] > 1.0:
            points[i, 0] -= 1.0
        if points[i, 1] < 0:
            points[i, 1] += 1.0
        elif points[i, 1] > 1.0:
            points[i, 1] -= 1.0
        
def step_chicken (points, partners, dt=DELTA_T):
    n = len (points)
    points_new = np.zeros (points.shape)
    for i in range (n):
        a = partners[i, 0] # aggressor
        d = partners[i, 1] # defender
        
        # Compute a "velocity" v for point i
        #v = ...
        # YOUR CODE HERE
        raise NotImplementedError()
        
        points_new[i, :] = points[i, :] + dt*v
        
    # Wraparound boundaries
    wrap (points_new)
    return points_new

In [ ]:
def sim (f_step, t_max):
    points = np.zeros ((t_max+1, points_0.shape[0], points_0.shape[1]))
    points[0, :, :] = points_0
    for t in range (t_max):
        points[t+1, :, :] = f_step (points[t, :, :], partners)
    return points

points_chicken = sim (step_chicken, 200)
plot_points_triplet (points_chicken[0, :, :], 'Before',
                     points_chicken[int (len (points_chicken)/2), :, :], "Middle",
                     points_chicken[-1, :, :], 'After')

In [ ]:
from ipywidgets import interact
def isim_chicken (t=0):
    plot_points (points_chicken[t, :, :])

interact (isim_chicken, t=(0, len (points_chicken)-1, 1))

**Exercise 2** (1 point). Now implement the "defender" behavior. That is, let $i$ be a defender point whose partners are $(a_i, b_i)$. Then $a_i$ is the aggressor, $b_i$ is the chicken, and $i$ will try to move toward the center of $a_i$ and $b_i$.

In [ ]:
def step_defender (points, partners, dt=DELTA_T):
    n = len (points)
    points_new = np.zeros (points.shape)
    for i in range (n):
        a = partners[i, 0] # aggressor
        c = partners[i, 1] # chicken
        
        # Compute a "velocity" v for the defender i
        # YOUR CODE HERE
        raise NotImplementedError()
        
        points_new[i, :] = points[i, :] + dt*v
        
    # Wraparound boundaries
    wrap (points_new)
    return points_new

In [ ]:
points_defender = sim (step_defender, 200)
plot_points_triplet (points_defender[0, :, :], 'Before',
                     points_defender[int (len (points_defender)/2), :, :], 'Middle',
                     points_defender[-1, :, :], 'After')

In [ ]:
def isim_defender (t=0):
    plot_points (points_defender[t, :, :])
    
interact (isim_defender, t=(0, len (points_defender)-1, 1)) ;

For a more dynamic demo of this simple model, see [The Game](http://www.icosystem.com/labsdemos/the-game/).